In [514]:
import os
import sys


SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["PYTHONHASHSEED"] = "42"
PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [515]:
from pyspark import SparkContext, SparkConf
import sys


conf = SparkConf()
sc = SparkContext(appName="hw_3_Uhbifg_1", conf=conf)


In [516]:
start_node = 1 # sys.argv[1]
end_node = 99 # sys.argv[2]
path_data = "/datasets/twitter/twitter.tsv"# sys.argv[3]
path_ans = "output_1"# sys.argv[4]

In [517]:
raw_graph = sc.textFile(path_data)
graph = raw_graph.map(lambda x: map(int, x.split("\t")[::-1])).cache()
links = graph.groupByKey().partitionBy(sc.defaultParallelism).mapValues(list).cache()     


In [ ]:
w = dict(links.collect())
def search(x1, x2, x3, seen):
    ans = list()
    a = x3[x1]
    for i in a:
        if i not in seen:
            ans.append(tuple([i, x2 + [i]]))
    return ans


links = graph.groupByKey().mapValues(list).cache()     
answers = []
pathes = sc.parallelize([(start_node, [start_node])])
seen = set([start_node])
num = 0
while True:
    if num == 500:
        ans = sc.parallelize([])
        break
    num += 1
    print(num, pathes.count())
    ans = pathes.filter(lambda x : x[0] == end_node)
    if ans.count() > 0:
        break
    else:
        rdd2 = pathes.map(lambda x : x[0], preservesPartitioning=True).distinct().collect()
        neigbours = dict() 
        for i in rdd2:
            neigbours[i] = w.get(i, [])
        pathes = sc.parallelize(pathes.flatMap(lambda x : search(x[0], x[1], neigbours, seen), preservesPartitioning=True).collect())
        seen.update(pathes.map(lambda x : x[0]).distinct().collect())
ans = ans.map(lambda x : x[1])

1 1
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0


In [ ]:
ans.collect()

In [432]:
a = [1]
a[-2]

IndexError: list index out of range

In [431]:
ans.collect()


[[12, 422, 53, 52, 107, 20, 23, 274, 34]]

In [392]:
path_ans = "1.csv"

In [397]:
ans = sc.parallelize([[1, 2, 3], [1, 3, 4]])

In [407]:
def toCSVLine(data):
    return ','.join(str(d) for d in data)

lines = ans.map(toCSVLine)
lines.saveAsTextFile(path_ans)

In [408]:
read_rdd = sc.textFile(path_ans + "1")


In [409]:
read_rdd.collect()

['1,2,3', '1,3,4']

In [405]:
lines.collect()

['1,2,3', '1,3,4']

In [394]:
!hdfs dfs -ls 

Found 10 items
drwx------   - Uhbifg Uhbifg          0 2021-03-15 06:00 .Trash
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-03 14:12 .hiveJars
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-29 23:43 .sparkStaging
drwx------   - Uhbifg Uhbifg          0 2021-02-24 14:46 .staging
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-29 23:58 1.csv
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-14 22:44 Uhbifg_hiveout
drwx------   - Uhbifg Uhbifg          0 2021-02-15 10:40 hwzero
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-29 23:55 output
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-03-29 23:56 output_1
drwxr-xr-x   - Uhbifg Uhbifg          0 2021-02-23 13:33 pred_with_filter


In [395]:
!hdfs dfs -cat 1.csv

cat: `1.csv': Is a directory


In [513]:
sc.stop()